## Weekly update 010521: addendum

In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px

from os import listdir
from plotly.subplots import make_subplots
from plotly import graph_objs as go, offline
from plotly.express.colors import qualitative

# Allow figures to work in HTML-exported version of notebook.
offline.init_notebook_mode()

### Training and validation results

Training and validation results regarding:

1. Accuracy
2. Recall
3. Precision
4. AUC
5. AUPR
6. Binary cross entropy loss

are read in and plotted here, so comparisons can be made between folds and between training and validation.

In [ ]:
# Constants for reading in CSVs and plotting.
PREFIXES = ["cv_1", "cv_2", "cv_3", "single"]
FOLDS = ["CV 1", "CV 2", "CV 3", "Single"]
FOLD_COLORS = dict(zip(FOLDS, qualitative.D3[:4]))

TRAINING_RESULTS_HEADER = [
    "Accuracy",
    "Recall",
    "Precision",
    "AUC",
    "AUPR",
    "BCE Loss",
    "roce_1",
    "roce_2",
    "roce_3",
    "roce_4",
]
VALIDATION_RESULTS_HEADER = ["Epoch"] + TRAINING_RESULTS_HEADER

In [ ]:
# Functions to read and compile CSVs.


def get_train_perf(prefix):
    """Return DataFrame of the specified fold's training performance."""
    curr_train_perf = pd.read_csv(
        f"train_test_stats/{prefix}_train_results.csv",
        header=None,
        names=TRAINING_RESULTS_HEADER,
    )
    curr_train_perf["Epoch"] = range(1, 31)
    curr_train_perf["Fold"] = (
        prefix.capitalize() if prefix == "single" else prefix.upper().replace("_", " ")
    )
    return curr_train_perf


def get_val_perf(prefix):
    """Return DataFrame of the specified fold's validation performance."""
    curr_val_perf = pd.read_csv(
        f"train_test_stats/{prefix}_test_results.csv",
        header=None,
        names=VALIDATION_RESULTS_HEADER,
    )
    curr_val_perf["Fold"] = (
        prefix.capitalize() if prefix == "single" else prefix.upper().replace("_", " ")
    )
    return curr_val_perf

In [ ]:
all_train_perf = pd.concat(
    [get_train_perf(prefix) for prefix in PREFIXES[:-1]],
    ignore_index=True,
)

all_val_perf = pd.concat(
    [get_val_perf(prefix) for prefix in PREFIXES],
    ignore_index=True,
)

In [ ]:
all_train_perf.head()

In [ ]:
all_val_perf.head()

#### More details on the validation schemes

Recall there are two validation schemes: the single validation scheme that uses the same proteins for training and validation, and the three-fold cross validation based on using data from three disjoint clusters of proteins, created to maxmize the difference between each cluster's proteins. More details on the exact nature of the clustering can be found in [Ragoze et al., 2017](https://pubmed.ncbi.nlm.nih.gov/28368587/).

*The proteins*

There are 102 proteins. The three clusters, and the validation folds they represent, are:

<table>
    <tr><th><center>CV 1</center></th><th><center>CV 2</center></th><th><center>CV 3</center></th></tr>
<tr><td>

| Name | Description |
|-:|:-|
| ABL1 | Tyrosine-protein kinase ABL |
| AKT1 | Serine/threonine-protein kinase AKT |
| AKT2 | Serine/threonine-protein kinase AKT2 |
| BRAF | Serine/threonine-protein kinase B-raf |
| CDK2 | Cyclin-dependent kinase 2 |
| CSF1R | Macrophage colony stimulating factor receptor |
| EGFR | Epidermal growth factor receptor erbB1 |
| FABP4 | Fatty acid binding protein adipocyte |
| FAK1 | Focal adhesion kinase 1 |
| FGFR1 | Fibroblast growth factor receptor 1 |
| GRIA2 | Glutamate receptor ionotropic, AMPA 2 |
| GRIK1 | Glutamate receptor ionotropic kainate 1 |
| HS90A | Heat shock protein HSP 90-alpha |
| IGF1R | Insulin-like growth factor I receptor |
| ITAL | Leukocyte adhesion glycoprotein LFA-1 alpha |
| JAK2 | Tyrosine-protein kinase JAK2 |
| KIF11 | Kinesin-like protein 1 |
| KIT | Stem cell growth factor receptor |
| KPCB | Protein kinase C beta |
| LCK | Tyrosine-protein kinase LCK |
| MAPK2 | MAP kinase-activated protein kinase 2 |
| MET | Hepatocyte growth factor receptor |
| MK01 | MAP kinase ERK2 |
| MK10 | c-Jun N-terminal kinase 3 |
| MK14 | MAP kinase p38 alpha |
| MP2K1 | Dual specificity mitogen-activated protein kinase kinase 1 |
| PLK1 | Serine/threonine-protein kinase PLK1 |
| ROCK1 | Rho-associated protein kinase 1 |
| SRC | Tyrosine-protein kinase SRC |
| TGFR1 | TGF-beta receptor type I |
| VGFR2 | Vascular endothelial growth factor receptor 2 |
| WEE1 | Serine/threonine-protein kinase WEE1 |
| XIAP | Inhibitor of apoptosis protein 3 |

</td><td>
    
| Name | Description |
|-:|:-|
| ACES | Acetylcholinesterase |
| ADA | Adenosine deaminase |
| ALDR | Aldose reductase |
| AMPC | Beta-lactamase |
| AOFB | Monoamine oxidase B |
| CAH2 | Carbonic anhydrase II |
| COMT | Catechol O-methyltransferase |
| DEF | Peptide deformylase |
| DHI1 | 11-beta-hydroxysteroid dehydrogenase 1 |
| DYR | Dihydrofolate reductase |
| FKB1A | FK506-binding protein 1A |
| FNTA | Protein farnesyltransferase/geranylgeranyltransferase type I alpha subunit |
| FPPS | Farnesyl diphosphate synthase |
| GLCM | Beta-glucocerebrosidase |
| HDAC2 | Histone deacetylase 2 |
| HDAC8 | Histone deacetylase 8 |
| HIVINT | Human immunodeficiency virus type 1 integrase |
| HIVRT | Human immunodeficiency virus type 1 reverse transcriptase |
| HMDH | HMG-CoA reductase |
| HXK4 | Hexokinase type IV |
| INHA | Enoyl-[acyl-carrier-protein] reductase |
| KITH | Thymidine kinase |
| NOS1 | Nitric-oxide synthase, brain |
| NRAM | Neuraminidase |
| PA2GA | Phospholipase A2 group IIA |
| PARP1 | Poly [ADP-ribose] polymerase-1 |
| PDE5A | Phosphodiesterase 5A |
| PGH1 | Cyclooxygenase-1 |
| PGH2 | Cyclooxygenase-2 |
| PNPH | Purine nucleoside phosphorylase |
| PTN1 | Protein-tyrosine phosphatase 1B |
| PUR2 | GAR transformylase |
| PYGM | Muscle glycogen phosphorylase |
| PYRD | Dihydroorotate dehydrogenase |
| SAHH | Adenosylhomocysteinase |
| TYSY | Thymidylate synthase |
    
</td><td>

| Name | Description |
|-:|:-|
| AA2AR | Adenosine A2a receptor |
| ACE | Angiotensin-converting enzyme |
| ADA17 | ADAM17 |
| ADRB1 | Beta-1 adrenergic receptor |
| ADRB2 | Beta-2 adrenergic receptor |
| ANDR | Androgen Receptor |
| BACE1 | Beta-secretase 1 |
| CASP3 | Caspase-3 |
| CP2C9 | Cytochrome P450 2C9 |
| CP3A4 | Cytochrome P450 3A4 |
| CXCR4 | C-X-C chemokine receptor type 4 |
| DPP4 | Dipeptidyl peptidase IV |
| DRD3 | Dopamine D3 receptor |
| ESR1 | Estrogen receptor alpha |
| ESR2 | Estrogen receptor beta |
| FA10 | Coagulation factor X |
| FA7 | Coagulation factor VII |
| GCR | Glucocorticoid receptor |
| HIVPR | Human immunodeficiency virus type 1 protease |
| LKHA4 | Leukotriene A4 hydrolase |
| MCR | Mineralocorticoid receptor |
| MMP13 | Matrix metalloproteinase 13 |
| PPARA | Peroxisome proliferator-activated receptor alpha |
| PPARD | Peroxisome proliferator-activated receptor delta |
| PPARG | Peroxisome proliferator-activated receptor gamma |
| PRGR | Progesterone receptor |
| RENI | Renin |
| RXRA | Retinoid X receptor alpha |
| THB | Thyroid hormone receptor beta-1 |
| THRB | Thrombin |
| TRY1 | Trypsin I |
| TRYB1 | Tryptase beta-1 |
| UROK | Urokinase-type plasminogen activator |

</td></tr> </table>

*The numbers*

We use a 1:1 active-to-decoys ratio for training, and a 1:50 ratio for validation. In the single validation scheme, an 80:20 training-to-validation ratio is used for splitting each protein's ligand data between training and validation.

| Fold | Actives | Decoys | Type |
|:-:|:-:|:-:|:-:|
| Single | 18204 | 18204 | Training |
| Single | 4601 | 230050 | Validation |
| CV 1 | 16392 | 16392 | Training |
| CV 1 | 6413 | 320650 | Validation |
| CV 2 | 16261 | 16261 | Training |
| CV 2 | 6544 | 327200 | Validation |
| CV 3 | 12957 | 12957 | Training |
| CV 3 | 9848 | 492400 | Validation |

### Training and validation comparison plots

I've chosen to display training and validation performance on different subplots with a shared x-axis because of great differences in y-axis scale, and to avoid overloading the same plot with multiple lines (for different training/validation mode-`Fold` combinations).

For the plot, I initially tried to use a Plotly Express line graph, but it doesn't seem to allow updating of the y-axis data. I've thus switched to using lower-level functions, which makes splitting data by fold more complex, and necessitates adding the traces of all metrics to the figure *before* plotting.

In [ ]:
def get_pivot_table(data, metric):
    """Return a table of `metric` values by fold from `data`."""
    return pd.pivot_table(data, values=metric, index=["Epoch"], columns="Fold")


def get_hovertext(metric, fold, epoch):
    """Return `metric` performance at epoch `epoch` of fold `fold` as hovertext."""
    # Find the position of the metric value, if any.
    text = f"<b>{fold} epoch {epoch}</b><br>"

    training_index = np.where(
        (all_train_perf.Epoch == epoch) & (all_train_perf.Fold == fold)
    )[0]
    if len(training_index) > 1:
        raise Exception(
            f"Training duplicates exist for {metric} in {fold} for epoch {epoch}."
        )
    elif len(training_index) == 1:
        text += f"Training: {all_train_perf[metric][training_index[0]].item():.3f}<br>"

    # If the epoch is even-numbered, validation may have been conducted for it.
    if epoch % 2 == 0:
        validation_index = np.where(
            (all_val_perf.Epoch == epoch) & (all_val_perf.Fold == fold)
        )[0]
        if len(validation_index) > 1:
            raise Exception(
                f"Validation duplicates exist for {metric} in {fold} for epoch {epoch}."
            )
        elif len(validation_index) == 1:
            text += f"Validation: {all_val_perf[metric][validation_index[0]].item():.3f}<br>"

    return text


def add_traces(fig, row, col, showlegend, data, metric, visible):
    """Return the figure after adding the traces of `metric` performance."""
    curr_data = get_pivot_table(data, metric)
    for fold in curr_data.columns:
        fig.add_trace(
            go.Scatter(
                x=curr_data.index,
                y=curr_data[fold].values,
                name=fold,
                mode="lines",
                line=dict(shape="linear", color=FOLD_COLORS[fold]),
                connectgaps=True,
                # Show only the legend of validation plots to avoid duplication.
                showlegend=showlegend,
                # Initially, only plots displaying accuracy are visible.
                visible=visible,
                hovertext=[
                    get_hovertext(metric, fold, epoch) for epoch in curr_data.index
                ],
                hoverinfo="text",
            ),
            row=row,
            col=col,
        )

In [ ]:
# Create facetted plot.
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.04,
    row_titles=["Training", "Validation"],
    x_title="Epoch",
    y_title="Accuracy",  # The initial displayed metric is accuracy.
)

# For each metric,
for metric in TRAINING_RESULTS_HEADER[:-4]:
    # add the training plots.
    add_traces(fig, 1, 1, False, all_train_perf, metric, metric == "Accuracy")
    # add the validation plots.
    add_traces(fig, 2, 1, True, all_val_perf, metric, metric == "Accuracy")

# Decouple the y-axes.
fig.update_yaxes(matches=None, title="")
# Couple together x-axes and set ticks.
fig.update_xaxes(
    tickmode="array",
    tickvals=list(range(2, np.max(all_train_perf.Epoch) + 1, 2)),
)

# Add button to change y-axis data.
fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            active=0,
            showactive=True,
            xanchor="left",
            yanchor="top",
            y=1.2,
            x=1,
            pad={"r": 10, "t": 10},
            buttons=list(
                [
                    dict(
                        label=metric,
                        method="update",
                        args=[
                            {
                                "visible": [
                                    k == i
                                    for k in range(len(TRAINING_RESULTS_HEADER[:-4]))
                                    for _ in range(7)
                                ]
                            },
                            {
                                "y": metric,
                                # Allow a dynamic y-axis title depending on chosen metric,
                                # while retaining fixed annotations.
                                "annotations": [
                                    {
                                        "font": {"size": 16},
                                        "showarrow": False,
                                        "text": "Training",
                                        "textangle": 90,
                                        "x": 0.98,
                                        "xanchor": "left",
                                        "xref": "paper",
                                        "y": 0.76,
                                        "yanchor": "middle",
                                        "yref": "paper",
                                    },
                                    {
                                        "font": {"size": 16},
                                        "showarrow": False,
                                        "text": "Validation",
                                        "textangle": 90,
                                        "x": 0.98,
                                        "xanchor": "left",
                                        "xref": "paper",
                                        "y": 0.24,
                                        "yanchor": "middle",
                                        "yref": "paper",
                                    },
                                    {
                                        "font": {"size": 16},
                                        "showarrow": False,
                                        "text": "Epoch",
                                        "x": 0.49,
                                        "xanchor": "center",
                                        "xref": "paper",
                                        "y": 0,
                                        "yanchor": "top",
                                        "yref": "paper",
                                        "yshift": -30,
                                    },
                                    {
                                        "font": {"size": 16},
                                        "showarrow": False,
                                        "text": metric,
                                        "textangle": -90,
                                        "x": 0,
                                        "xanchor": "right",
                                        "xref": "paper",
                                        "xshift": -40,
                                        "y": 0.5,
                                        "yanchor": "middle",
                                        "yref": "paper",
                                    },
                                ],
                            },
                        ],
                    )
                    for i, metric in enumerate(TRAINING_RESULTS_HEADER[:-4])
                ]
            ),
        ),
    ],
)

fig.show()

As can be seen, the training performance across folds and across metrics is near-perfect within 2-4 epochs. It is thus divorced from validation performance, particularly for `CV 3`, which has markedly worse precision than models trained under different folds. Moreover, we notice the following:

1. Performance on `CV 3` in terms of the metrics it lags in - primarily precision and loss - show improvement towards the very last few epochs. We should train the model on more epochs and then re-validate it. This is in stark contrast to the model's performance in other folds, where it achieves peak (AUPR) performance within <10 epochs. (Note that in our meeting last week, the last two epoch's worth of validation data seem to have been cut off from the visualization due to a bug with my graphing implementation; thanks for asking me to re-plot things again in less of a rush.)
2. Performance of the model on other folds, while generally better, is still quite unstable in terms of precision.
3. As noted before, AUC is a poor metric for model performance because it does not reflect precision even indirectly, given the high active-to-decoys ratio.

From both 1. and 2., we see that our learning procedure may have room for improvement: should we make the model more shallow? Should we use a different optimizer (we are currently using Adam with a small initial learning rate)? We need to stabilize learning and get it taking "good learning directions" consistently and in few epochs.

Plotly includes the metric selection button when the image is exported. I couldn't find a way to hide it during export, and so have created the function below to export the plot for each specific metric:

In [ ]:
# Functions to write training/validation plots to disk.


def get_fig(metric):
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.04,
        row_titles=["Training", "Validation"],
        x_title="Epoch",
        y_title=metric,
    )

    # Add the training plots.
    add_traces(fig, 1, 1, False, all_train_perf, metric, True)
    # Add the validation plots.
    add_traces(fig, 2, 1, True, all_val_perf, metric, True)

    fig.update_yaxes(matches=None, title="")
    fig.update_xaxes(
        tickmode="array",
        tickvals=list(range(2, np.max(all_train_perf.Epoch) + 1, 2)),
    )

    return fig


def write_static_image(metric, path):
    """Write training/validation plot for `metric` as static image."""
    get_fig(metric).write_image(path)


def write_all_static_images(base_path, ext):
    """Write training/validation plot for all metrics as static images."""
    for metric in TRAINING_RESULTS_HEADER[:-4]:
        write_static_image(metric, f"{base_path}/{metric}.{ext}")


def write_html(metric, path):
    """Write training/validation plot for `metric` to HTML."""
    get_fig(metric).write_html(path, include_plotlyjs="cdn")


def write_all_htmls(base_path):
    """Write all training/validation plots to HTML."""
    for metric in TRAINING_RESULTS_HEADER[:-4]:
        write_html(metric, f"{base_path}/{metric}.html")

In [ ]:
write_all_static_images("images", "pdf")
write_all_htmls("images")

### More details about the data and model

#### Ligand SMILES representation

Ligands' SMILES sequences are encoded using a bi-directional LSTM. Here, we consider the sequences' lengths (simply, the number of characters in each SMILES sequence).

In [ ]:
# Functions to get the lengths of SMILES sequences.


def get_length(path):
    """Return lengths of ligands in file `path`."""
    with open(path, "r") as f:
        return [len(line.split()[0]) for line in f.readlines()]


def get_all_lengths(folder_path):
    """Return list with the folder's ligand sequence lengths."""
    return [
        length
        for file in listdir(folder_path)
        for length in get_length(f"{folder_path}/{file}")
        if file.endswith(".ism")
    ]

In [ ]:
actives_lengths = get_all_lengths("dude_data/actives")
decoys_lengths = get_all_lengths("dude_data/decoys")

actives_median = np.median(actives_lengths)
decoys_median = np.median(decoys_lengths)

x_axis_ticks = list(range(0, 200, 20)) + [
    actives_median,
    decoys_median,
]
x_axis_ticks.remove(60)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=actives_lengths,
        histnorm="probability density",
        name="Actives",
        hovertemplate="Length: %{x}<br>Density: %{y}",
    ),
)
fig.add_trace(
    go.Histogram(
        x=decoys_lengths,
        histnorm="probability density",
        name="Decoys",
        hovertemplate="Length: %{x}<br>Density: %{y}",
    )
)

# Overlay both histograms
fig.update_layout(
    barmode="overlay",
    title="Ligand SMILES sequence lengths",
    xaxis=dict(
        title="Sequence length",
        tickmode="array",
        tickvals=x_axis_ticks,
    ),
    yaxis=dict(title="Probability density"),
    # Mean lines
    shapes=[
        {
            "line": {"color": qualitative.Plotly[0], "dash": "solid", "width": 3},
            "type": "line",
            "x0": actives_median,
            "x1": actives_median,
            "xref": "x",
            "y0": -0.005,
            "y1": 1,
            "yref": "paper",
        },
        {
            "line": {"color": qualitative.Plotly[1], "dash": "solid", "width": 3},
            "type": "line",
            "x0": decoys_median,
            "x1": decoys_median,
            "xref": "x",
            "y0": -0.005,
            "y1": 1,
            "yref": "paper",
        },
    ],
)

# Reduce opacity to see both histograms.
fig.update_traces(opacity=0.75)
fig.show()

While the decoys associated with each active are chosen to have "similar physico-chemical properties but dissimilar 2-D topology" [(DUD-E website)](http://dude.docking.org/), the sequence lengths specifically are quite similar between the two groups, but with a few decoys being significantly larger than the rest.

Moreover, within both groups, we see quite a large spread of lengths. While LSTMs can be effective in encoding long sequences, we should perhaps consider: how does sequence length relate to model performance? If the two are related significantly, what can we do to further improve performance?

#### Protein contact map representation

Proteins are represented using the contact map of their residues, and the contact map is processed using a 2D-CNN. The following is a distribution of the number of residues per protein:

In [ ]:
# Functions to get the number of residues per protein.


def get_length(path):
    """Return number of residues in file `path`."""
    with open(path, "r") as f:
        return len(f.readlines()) - 2


all_protein_lengths = [
    get_length(f"dude_data/contact_maps/{file}")
    for file in listdir("dude_data/contact_maps")
    if file.endswith("_full")
]

num_residues_median = np.median(all_protein_lengths)
x_axis_ticks = list(range(100, 900, 100)) + [num_residues_median]
x_axis_ticks.remove(300)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=all_protein_lengths,
        histnorm="probability density",
        hovertemplate="Residues: %{x}<br>Density: %{y}",
        nbinsx=40,
    ),
)

# Overlay both histograms
fig.update_layout(
    title="Number of residues per protein",
    xaxis=dict(
        title="Number of residues",
        tickmode="array",
        tickvals=x_axis_ticks,
    ),
    yaxis=dict(title="Probability density"),
    # Mean lines
    shapes=[
        {
            "line": {"color": "black", "dash": "dash", "width": 2},
            "type": "line",
            "x0": num_residues_median,
            "x1": num_residues_median,
            "xref": "x",
            "y0": -0.005,
            "y1": 1,
            "yref": "paper",
        },
    ],
)

# Reduce opacity to see both histograms.
fig.update_traces(opacity=0.75)
fig.show()

The 2D-CNN used to encode protein contact maps consists of eight stacked residual blocks and a sequential self-attention block:

<img src="images/residual_block.png" width="375"/>

Given contact maps can be of different sizes, the residual blocks do not enforce any dimensionality restrictions. Rather, after channelization through these blocks, the processed representation is pooled along each channel within the attention block to a fixed-size representation.

### To do

From the diagnosis above, we need to consider:

1. how does ligand sequence length relate to model performance?
2. given training looks all-OK, how can we get more stable validation performance?